In [ ]:
import numpy as np

import tensorflow as tf

from keras.models import Sequential, load_model
from keras.layers import Dense, Activation
from keras.layers import LSTM, Dropout
from keras.layers import TimeDistributed
from keras.layers.core import Dense, Activation, Dropout, RepeatVector
from keras.optimizers import RMSprop
import pickle
import sys
import heapq
from pylab import rcParams

In [ ]:
# find the total length 
path = 'nietzsche.txt'
text = open(path).read().lower()
print('corpus length:', len(text))

corpus length: 600893


In [ ]:
# unique characters
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))
print(f'unique chars: {len(chars)}')

unique chars: 57


In [ ]:
# splitting into sequences of 40 characters
SEQUENCE_LENGTH = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - SEQUENCE_LENGTH, step):
    sentences.append(text[i: i + SEQUENCE_LENGTH])
    next_chars.append(text[i + SEQUENCE_LENGTH])
print(f'num training examples: {len(sentences)}')

num training examples: 200285


In [ ]:
X = np.zeros((len(sentences), SEQUENCE_LENGTH, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [ ]:
sentences[100]

've been unskilled and unseemly methods f'

In [ ]:
next_chars[100]

'o'

In [ ]:
X[0][0],y[0]


(array([False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False,  True, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False]),
 array([False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False,  True, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False]))

In [ ]:
X.shape,y.shape

((200285, 40, 57), (200285, 57))

In [ ]:
model = Sequential()
model.add(LSTM(128, return_sequences=False,input_shape=(SEQUENCE_LENGTH, len(chars))))
#model.add(Dropout(0.2))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

In [ ]:
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer= optimizer, metrics=['accuracy'])
history = model.fit(X, y, validation_split=0.05, batch_size=128, epochs=20, shuffle=True).history

Epoch 1/20
1487/1487 [==============================] - 179s 121ms/step - loss: 1.9868 - accuracy: 0.4181 - val_loss: 1.6705 - val_accuracy: 0.4984
Epoch 2/20
1487/1487 [==============================] - 181s 122ms/step - loss: 1.6266 - accuracy: 0.5162 - val_loss: 1.5685 - val_accuracy: 0.5284
Epoch 3/20
1487/1487 [==============================] - 177s 119ms/step - loss: 1.5371 - accuracy: 0.5404 - val_loss: 1.5313 - val_accuracy: 0.5388
Epoch 4/20
1487/1487 [==============================] - 185s 125ms/step - loss: 1.4893 - accuracy: 0.5542 - val_loss: 1.5284 - val_accuracy: 0.5373
Epoch 5/20
1487/1487 [==============================] - 179s 121ms/step - loss: 1.4599 - accuracy: 0.5613 - val_loss: 1.5001 - val_accuracy: 0.5444
Epoch 6/20
1487/1487 [==============================] - 179s 120ms/step - loss: 1.4356 - accuracy: 0.5676 - val_loss: 1.4964 - val_accuracy: 0.5474
Epoch 7/20
1487/1487 [==============================] - 181s 122ms/step - loss: 1.4177 - accuracy: 0.5727 - val_

In [ ]:
model.save('keras_model.h5')
pickle.dump(history, open("history.p", "wb"))

In [ ]:
model = load_model('keras_model.h5')
history = pickle.load(open("history.p", "rb"))

In [ ]:
def prepare_input(text):
    x = np.zeros((1, SEQUENCE_LENGTH, len(chars)))
    for t, char in enumerate(text):
        x[0, t, char_indices[char]] = 1.
    return x

In [ ]:
prepare_input("This is an example of input for our LSTM".lower())

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

In [ ]:
# next possible characters
def sample(preds, top_n=3):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    return heapq.nlargest(top_n, range(len(preds)), preds.take)

In [ ]:
# predict next character until space
def predict_completion(text):
    original_text = text
    generated = text
    completion = ''
    while True:
        x = prepare_input(text)
        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, top_n=1)[0]
        next_char = indices_char[next_index]
        text = text[1:] + next_char
        completion += next_char
        if len(original_text + completion) + 2 > len(original_text) and next_char == ' ':
            return completion

In [ ]:
def predict_completions(text, n=3):
    x = prepare_input(text)
    preds = model.predict(x, verbose=0)[0]
    next_indices = sample(preds, n)
    return [indices_char[idx] + predict_completion(text[1:] + indices_char[idx]) for idx in next_indices]

In [ ]:
quotes = [
    "It is not a lack of love, but a lack of friendship that makes unhappy marriages.",
    "That which does not kill us makes us stronger.",
    "I'm not upset that you lied to me, I'm upset that from now on I can't believe you.",
    "And those who were seen dancing were thought to be insane by those who could not hear the music.",
    "It is hard enough to remember my opinions, without also remembering my reasons for them!"
]

In [ ]:
for q in quotes:
    seq = q[:40].lower()
    print(seq)
    print(predict_completions(seq, 5))
    print()

it is not a lack of love, but a lack of 
['the ', 'a ', 'his ', 'some ', 'interpret ']

that which does not kill us makes us str
['ongest ', 'ange ', 'ength ', 'ive ', 'uggler ']

i'm not upset that you lied to me, i'm u
['niversal ', 's ', 'pon ', 'tility ', 'ltimate ']

and those who were seen dancing were tho
['ugh ', 'se ', 'rought ', 'ots ', 'm ']

it is hard enough to remember my opinion
[' of ', ', ', '\nand ', 's ', '. ']



In [ ]:
#THANK YOU.....